In [1]:
import random
import os
os.environ["OMP_NUM_THREADS"] = "1"
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import torch
import cv2
import warnings
import timeit
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from torch import optim
from scipy.special import expit
from skimage import io, transform, measure
from sklearn import metrics
import optuna
from config import *
from util import *
from models import *
sns.set()
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
rc = {"figure.figsize" : (9, 7),
      "axes.spines.left" : False,
      "axes.spines.right" : False,
      "axes.spines.bottom" : False,
      "axes.spines.top" : False,
      "xtick.bottom" : True,
      "xtick.labelbottom" : False,
      "ytick.labelleft" : False,
      "ytick.left" : True,
      "axes.grid" : False}
plt.rcParams.update(rc)

In [2]:
best_accuracy = 0

def objective(trial):
    torch.cuda.empty_cache()
    random.seed(seed)

    params = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 5e-4),
        'dropout': trial.suggest_float('dropout', 0.3, 0.7),
        'batch_size': trial.suggest_categorical('batch_size', [16, 32])
    }
    
    checkpoint_callback = ModelCheckpoint(save_top_k=1, monitor="val/loss")
    model = BaselineClassifier(img_dir=img_dir, metadata_file=metadata_file, weighted_sampling=False,
                               batch_size=params['batch_size'], dropout=params['dropout'], learning_rate=params['learning_rate'])

    trainer = pl.Trainer(max_epochs=25, devices=[1], accelerator="gpu", deterministic=True, callbacks=[checkpoint_callback], weights_summary=None)
    trainer.fit(model)
    
    # Load the best checkpoint after training
    best_checkpoint_path = checkpoint_callback.best_model_path
    checkpoint = torch.load(best_checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])
    
    result_val = get_predictions(trainer, model, split='val', dx_threshold=0)
    dx_threshold = find_optimal_cutoff(result_val['benign_malignant'], result_val['dx_score'])
    result_test = get_predictions(trainer, model, split='test', dx_threshold=dx_threshold)
    result_external = get_predictions(trainer, model, split='external', dx_threshold=dx_threshold) 
    
    val_accuracy = metrics.balanced_accuracy_score(result_val['benign_malignant'], result_val['prediction'])
    test_accuracy = metrics.balanced_accuracy_score(result_test['benign_malignant'], result_test['prediction'])
    external_accuracy = metrics.balanced_accuracy_score(result_external['benign_malignant'], result_external['prediction'])
    
    trial.set_user_attr("val_accuracy", val_accuracy)
    trial.set_user_attr("test_accuracy", test_accuracy)
    trial.set_user_attr("external_accuracy", external_accuracy)
    
    global best_accuracy
    if val_accuracy >= best_accuracy:
        best_accuracy = val_accuracy
        torch.save(model.state_dict(), os.path.join("models", "baseline_model.pth"))
        result_val.to_csv(os.path.join("results", "result_val.csv"), index=False)
        result_test.to_csv(os.path.join("results", "result_test.csv"), index=False)
        result_external.to_csv(os.path.join("results", "result_external.csv"), index=False)
    
    return val_accuracy

study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=seed), storage="sqlite:///optuna/baseline_model.db")
study.optimize(objective, n_trials=15, gc_after_trial=True)

[I 2023-09-04 10:03:19,329] A new study created in RDB with name: no-name-ea9cc6d0-8039-48d9-ac8f-25226c176ce3
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting: 169it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting: 169it [00:00, ?it/s]

[I 2023-09-04 12:10:11,617] Trial 14 finished with value: 0.8294268406337372 and parameters: {'learning_rate': 1.938368339476709e-05, 'dx_dropout': 0.43225635109359833, 'batch_size': 16}. Best is trial 4 with value: 0.8440354147250699.


In [4]:
print(study.best_params)
print(study.best_value)
trials_df = study.trials_dataframe().sort_values('value', ascending=False)
display(trials_df)
trials_df.to_csv(os.path.join('optuna', 'baseline_model.csv'), index=False)

{'batch_size': 16, 'dx_dropout': 0.5099025726528951, 'learning_rate': 3.2877474139911175e-05}
0.8440354147250699


,number,value,datetime_start,datetime_complete,duration,params_batch_size,params_dx_dropout,params_learning_rate,user_attrs_external_accuracy,user_attrs_test_accuracy,user_attrs_val_accuracy,state
4,4,0.844035,2023-09-04 10:36:11.684251,2023-09-04 10:44:45.179623,0 days 00:08:33.495372,16,0.509903,0.000033,0.763966,0.856115,0.844035,COMPLETE
0,0,0.841333,2023-09-04 10:03:19.341295,2023-09-04 10:11:51.890674,0 days 00:08:32.549379,16,0.680286,0.000043,0.789064,0.851330,0.841333,COMPLETE
13,13,0.836510,2023-09-04 11:53:01.461913,2023-09-04 12:01:38.214714,0 days 00:08:36.752801,16,0.535529,0.000057,0.753453,0.832153,0.836510,COMPLETE
2,2,0.831174,2023-09-04 10:19:56.641465,2023-09-04 10:28:03.997518,0 days 00:08:07.356053,32,0.583229,0.000105,0.791641,0.836191,0.831174,COMPLETE
14,14,0.829427,2023-09-04 12:01:38.421662,2023-09-04 12:10:11.604893,0 days 00:08:33.183231,16,0.432256,0.000019,0.778705,0.826545,0.829427,COMPLETE
9,9,0.828332,2023-09-04 11:18:17.827566,2023-09-04 11:26:57.492403,0 days 00:08:39.664837,16,0.339069,0.000033,0.774479,0.857761,0.828332,COMPLETE
11,11,0.828332,2023-09-04 11:35:41.341368,2023-09-04 11:44:22.085522,0 days 00:08:40.744154,16,0.505783,0.000039,0.756030,0.844230,0.828332,COMPLETE
8,8,0.818173,2023-09-04 11:09:36.506394,2023-09-04 11:18:17.641362,0 days 00:08:41.134968,16,0.679554,0.000013,0.771233,0.837761,0.818173,COMPLETE
3,3,0.817591,2023-09-04 10:28:04.189462,2023-09-04 10:36:11.493268,0 days 00:08:07.303806,32,0.384936,0.000260,0.771233,0.828191,0.817591,COMPLETE
5,5,0.817521,2023-09-04 10:44:45.360023,2023-09-04 10:52:51.811191,0 days 00:08:06.451168,32,0.355798,0.000110,0.759070,0.793053,0.817521,COMPLETE
